# Search SWOT passes

This notebook is similar to the notebook [main](main.ipynb) but it uses the the
public API of the module to search for SWOT passes using the programmatic
interface instead of the
[voila](https://voila.readthedocs.io/en/stable/using.html) interface.

First, we import the module and create the application object. It is not
mandatory to create the application object but it is useful to have access to
the GUI to create the bounding box.

In [ ]:
from package.widgets import MapSelection, compute_selected_passes
from package.orbit import get_selected_passes, get_pass_passage_time

app = MapSelection()

Display the GUI to create the bounding box.

In [ ]:
app.display()

The easiest way to get the selected passes is to use the method provided by the application.

In [ ]:
import pyinterp
import warnings

if app.selection is None:
    warnings.warn('No selection has been done with the GUI, '
                  'a default selection is used')
    app.selection = pyinterp.geodetic.Polygon.read_wkt(
        'POLYGON((-48 -12,-48 42,0 42,0 -12,-48 -12))')
df = compute_selected_passes(app.date_selection, app)
df

To save the result in a CSV file execute: `df.to_csv("passes.csv")`

## API

An other way to get the selected passes is to use the function provided by the package.
The following code computes the selected passes for the next 72 hours. The first
parameter is the current time, the second parameter is the time interval.

*Adjust the parameters as needed.*


In [ ]:
import numpy

df = get_selected_passes(numpy.datetime64('now'), numpy.timedelta64(3, 'D'))
df

Now we compute the passage time for the selected passes for a given polygon.
Here you have two options:

1. Use the polygon defined in the app : `app.selection`
2. Define your own polygon, using the
   [WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry)
   format.
3. Define your own polygon, using [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON)

In [ ]:
bbox = pyinterp.geodetic.Polygon.read_wkt(
    'POLYGON((-6 36,-6 60,36 60,36 36,-6 36))')
bbox

In [ ]:
import pandas

df = pandas.DataFrame(get_pass_passage_time(selected_passes=df, polygon=bbox))
df

## To display the SWATH with matplotlib

The last section of this notebook shows how to plot the selected passes with
matplotlib.

In [ ]:
from package.widgets import load_polygons

In [ ]:
left_swath, right_swath = load_polygons(df['pass_number'].values)

In [ ]:
import cartopy.crs
import matplotlib.pyplot
import matplotlib
import matplotlib.patches
import matplotlib.colors
import matplotlib.cm

color_norm = matplotlib.colors.Normalize(vmin=0,
                                         vmax=df['pass_number'].values.max())
color_map = matplotlib.cm.ScalarMappable(norm=color_norm, cmap='jet')

fig = matplotlib.pyplot.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=cartopy.crs.PlateCarree())
ax.coastlines()
ax.gridlines()

points = bbox.outer
lons = numpy.array([p.lon for p in points])
lats = numpy.array([p.lat for p in points])
labels = {}

ax.plot(lons, lats, transform=cartopy.crs.Geodetic(), color='red')
for pass_number, item in left_swath:
    item = item.intersection(bbox)
    if len(item) == 0:
        continue
    outer = item[0].outer
    poly = matplotlib.patches.Polygon([(p.lon, p.lat) for p in outer],
                                      transform=cartopy.crs.PlateCarree(),
                                      facecolor=color_map.to_rgba(pass_number),
                                      alpha=0.5)
    labels[pass_number] = True
    poly.set_label(f'Pass {pass_number}')
    ax.add_patch(poly)

for pass_number, item in right_swath:
    item = item.intersection(bbox)
    if len(item) == 0:
        continue
    outer = item[0].outer
    poly = matplotlib.patches.Polygon([(p.lon, p.lat) for p in outer],
                                      transform=cartopy.crs.PlateCarree(),
                                      facecolor=color_map.to_rgba(pass_number),
                                      alpha=0.5)
    if pass_number not in labels:
        poly.set_label(f'Pass {pass_number}')
    ax.add_patch(poly)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='lower left')